# Word2Vec Exercises

## Introduction

We will be using 2 well-known text datasets to explore the capabilities of Word2Vec:
- [Spam Classification Dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/): Collection of SMS text messages labeled as Spam/Not Spam (Ham)
- [20 Newsgroups Dataset](http://qwone.com/~jason/20Newsgroups/): Famous text classification dataset from user discussion forums with 20 classes, including...
  - Computer Graphics (comp.graphics)
  - Microsoft Windows (comp.os.ms-windows.misc)
  - IBM Hardware (comp.sys.ibm.pc.hardware)
  - Mac Hardware (comp.sys.mac.hardware)
  - Windows XP (comp.windows.x)
  - For Sale (misc.forsale)
  - Automobiles (rec.autos)
  - Motorcycles (rec.motorcycles)
  - Baseball (rec.sport.baseball)
  - Hockey (rec.sport.hockey)
  - General Politics (talk.politics.misc)
  - Politics - Gun Control (talk.politics.guns)
  - Middle East Politics (talk.politics.mideast)
  - Cryptography (sci.crypt)
  - Electronics (sci.electronics)
  - Medicine (sci.med)
  - Space (sci.space)
  - Religion (talk.religion.misc)
  - Atheism (alt.atheism)
  - Christianity (soc.religion.christian)
  
To perform our tasks, we will both derive our own **word vectors** from the data as well as borrow Google's massive set of word vectors trained on the web ([Google Vectors](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)). Download these and put them under nltk data - note that this file is pretty large!

In [1]:
# NLP tools
import nltk
import gensim

# Data tools
import numpy as np
import pandas as pd

# Necessary for adding accessory_functions module to path
import os, sys
lib_path = os.path.abspath(os.path.join('..', '..'))
sys.path.append(lib_path)
from accessory_functions import google_vec_file, nltk_path

# Python 2 compatibility
from __future__ import print_function

# Setup nltk corpora path
nltk.data.path.insert(0, nltk_path)

## Question 1
**Loading Google Word2Vec Vectors**
* Load the Google vectors into an object `google_model` using `gensim` (This step will take awhile, as it has to load 3 million vectors into the appropriate Word2Vec format).
* Confirm that you have 3 million vectors of length 300.

In [2]:
# Load the Google vectors
google_model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

Google's model contains an extensive vocabulary.

In [3]:
type(google_model.vocab)

dict

In [4]:
# Number of Vectors
len(google_model.vocab.keys())

3000000

In [5]:
# Size of the Vectors
google_model.vector_size

300

## Question 2
**Exploring Word2Vec Vectors**
* Print out a few word vectors from the Google set
* Print out the similarity between the following pairs (feel free to experiment with more if you like):
  * baseball, bat
  * baseball, ocean
  * bat, fly
* What sorts of patterns do you notice?  Where does it succeed?  Where does it fail?  How might one improve it?
* Print out the most similar words to the following words:
  * baseball
  * president
* Print out words similar to the positive words and dissimilar to the negative words for the following positive/negative groups:
* Print out the words that don't match the others in each of the following groups:

In [6]:
# Word Vectors
google_model.word_vec('baseball')

array([-0.10546875,  0.25585938,  0.25390625, -0.03125   , -0.28515625,
       -0.1953125 ,  0.18261719, -0.06298828,  0.18457031,  0.08007812,
       -0.07324219, -0.25585938, -0.12792969,  0.08251953, -0.046875  ,
       -0.08984375,  0.07714844,  0.3515625 ,  0.26367188, -0.13867188,
       -0.125     ,  0.09130859, -0.01062012, -0.140625  , -0.04321289,
        0.15917969, -0.07763672,  0.34179688,  0.12255859,  0.30273438,
        0.04956055,  0.140625  , -0.0378418 ,  0.04052734, -0.01831055,
        0.07128906,  0.29492188, -0.17773438,  0.2578125 ,  0.18652344,
        0.18261719, -0.07128906,  0.22558594, -0.05932617,  0.05566406,
        0.17773438,  0.1875    , -0.1875    , -0.05664062,  0.32421875,
       -0.05297852,  0.00236511, -0.08398438, -0.0123291 , -0.03088379,
       -0.16015625, -0.01043701, -0.06835938, -0.01257324, -0.16601562,
       -0.11376953, -0.0612793 ,  0.13183594,  0.07324219,  0.23730469,
        0.00515747, -0.23535156,  0.13378906,  0.2734375 ,  0.08

In [7]:
# Pairwise Similarity
google_model.similarity('Bill_Clinton', 'Barack_Obama')

0.62116989722645277

* Word Sense Disambiguation

In [8]:
# Most similar words
google_model.similar_by_word('Barack_Obama')

[('Obama', 0.8036513328552246),
 ('Barrack_Obama', 0.7766816020011902),
 ('Illinois_senator', 0.757197916507721),
 ('McCain', 0.7530534267425537),
 ('Barack', 0.7448184490203857),
 ('Barack_Obama_D-Ill.', 0.7196038961410522),
 ('Hillary_Clinton', 0.6864978671073914),
 ('Sen._Hillary_Clinton', 0.6827855110168457),
 ('elect_Barack_Obama', 0.6812860369682312),
 ('Clinton', 0.6713167428970337)]

In [9]:
# Positive Negative Similar Words
google_model.most_similar(positive=['Barack_Obama', 'Bill_Clinton'], negative=['president'])
google_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=3)

[('queen', 0.7118192315101624),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951)]

In [10]:
# Words that don't match
google_model.doesnt_match(['Mitt_Romney', 'Ted_Cruz', 'Hillary_Clinton', 'John_McCain', 'President'])

'President'

## Question 3
**Document Vectors from Word Vectors**
* Compare the following sample documents to each other by cosine similarity
* Compare the same set of documents together by Word Mover's Distance  

***Note***: You will need to take care to first do the following:
  * Split the documents into lists of words.
  * Remove all words that aren't in the Google vector vocabulary (Word2Vec errors otherwise).

In [11]:
# Comparing via Cosine Similarity
google_model.n_similarity(['Barack_Obama', 'president'], ['Bill_Clinton', 'President'])

0.72718535909185844

In [12]:
#!pip install pyemd

In [13]:
# Comparing via Word Mover's Distance
google_model.wmdistance(['Barack_Obama', 'president'], ['Bill_Clinton', 'President'])

2.283404423946738

## Vocabulary Features

Each word contains an array of 300 features.

In [14]:
len(google_model.word_vec('cat'))

300

In [15]:
google_model.word_vec('cat')[:20]

array([ 0.0123291 ,  0.20410156, -0.28515625,  0.21679688,  0.11816406,
        0.08300781,  0.04980469, -0.00952148,  0.22070312, -0.12597656,
        0.08056641, -0.5859375 , -0.00445557, -0.296875  , -0.01312256,
       -0.08349609,  0.05053711,  0.15136719, -0.44921875, -0.0135498 ], dtype=float32)

The cosine similarity between words can be computed and produces intuitive trends.

In [16]:
print(google_model.similarity('cat', 'cat'))
print(google_model.similarity('cat', 'dog'))
print(google_model.similarity('cat', 'car'))

1.0
0.760945708978
0.215281850364


In [17]:
print(google_model.similarity('car', 'truck'))
print(google_model.similarity('car', 'drive'))

0.673579016963
0.313944691624


Word2Vec captures some interesting similarities between words, such as the relationship between **man --> king** and **woman --> queen**.

In [18]:
google_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=3)

[('queen', 0.7118192315101624),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951)]

In [19]:
google_model.wmdistance("Obama is the president of the United States".lower().split(), 
                        "Bush was the president of the United States".lower().split())

0.8887498780166667

##### Question: Why does the previous command take so much longer than others?
Because it has to generate a new vector that is **woman** + **king** - **man** and compare that vector to all 3 million vectors, then sort to find the closest 3.  The 3 million are stored in such a way that they can be compared quickly, but any new vector is not.

It can also detect words that don't belong in a sequence.

In [20]:
google_model.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

## Question 4
**Training a Word2Vec Model**
* Read the spam dataset into a Dataframe.
* Preprocess the documents using our previous `accesory_functions`.
* Split each document into a list of words.
* Use the results to train your own word2vec model with `gensim`.

##### Question: What does "training a word2vec model" mean?
Answer: generating good word vectors, or "word embeddings" for all unique words in the text dataset.  "Good" means similar words have their vectors cluster close together.

In [21]:
import pandas as pd
from accessory_functions import preprocess_series_text, nltk_path

spam_data = pd.read_csv('data/spam.csv', sep='\t', header=None)
spam_data.columns = ['label','text']
spam_data['text'] = preprocess_series_text(spam_data.text, nltk_path=nltk_path)

spam_data.head()

,label,text
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry wkly comp win fa cup final tkts may...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though


Now let's train the Word2Vec model.  `gensim` requires the documents to be represented as a ***list of sentences*** to train Word2Vec.  Here we'll do this by calling `split()` on our document text to turn each document into a list of words.

In [22]:
from gensim.models import Word2Vec
# Generate sentences for training word2vec
sentences = spam_data.text.str.split()
# Train a Word2Vec model
spam_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

Check out some conceptual comparisons with our word2vec model.

In [23]:
spam_model.most_similar('call')

[('please', 0.9998553991317749),
 ('customer', 0.9998331665992737),
 ('number', 0.9998165369033813),
 ('service', 0.9998107552528381),
 ('reply', 0.9998052716255188),
 ('holiday', 0.9998008608818054),
 ('text', 0.999797523021698),
 ('show', 0.9997959733009338),
 ('line', 0.9997928738594055),
 ('live', 0.999782383441925)]

Not bad, but if you investigate deeper you realize the results are so-so at best:

In [24]:
spam_model.most_similar('love')

[('life', 0.9999260902404785),
 ('dont', 0.9999098181724548),
 ('time', 0.9999085068702698),
 ('make', 0.9999074935913086),
 ('say', 0.9999046325683594),
 ('one', 0.9999046325683594),
 ('day', 0.999902069568634),
 ('well', 0.9999009370803833),
 ('get', 0.9998995661735535),
 ('heart', 0.9998987913131714)]

In [25]:
spam_model.most_similar('start')

[('get', 0.9999110102653503),
 ('see', 0.999899685382843),
 ('already', 0.9998975396156311),
 ('b', 0.9998968243598938),
 ('thing', 0.9998928308486938),
 ('v', 0.9998915195465088),
 ('time', 0.9998908638954163),
 ('da', 0.9998903274536133),
 ('month', 0.9998899698257446),
 ('person', 0.9998892545700073)]

Word2Vec requires **A LOT** of data to get a really good set of vectors.  Thankfully, as you saw above, Google (and others) have done this work for you, and you can usually just load in their vectors for your tasks.

## Word2Vec for Machine Learning

As you know, once we have vectors for examples, we can perform Machine Learning (both supervised and unsupervised).

#### From Word Vectors to Document Vectors
Consider the case of document classification.  From Word2Vec we have vectors for words, but our examples to classify are documents.  

How do we get vectors for whole documents?

The most common answer is to take an average of all the word vectors in a document.  Let's try that with our spam data.

In [26]:
# Function to take a document as a list of words and return the document vector
def get_doc_vec(words, model):
    good_words = []
    for word in words:
        # Words not in the original model will fair
        try:
            if model[word] is not None:
                good_words.append(word)
        except:
            continue
    # If no words are in the original model
    if len(good_words) == 0:
        return None
    # Return the mean of the vectors for all the good words
    return model[good_words].mean(axis=0)

Use our function to generate the document vectors for our spam data.

In [27]:
# Make a copy of the data to not disturb the original
spam_data1 = spam_data.copy()
spam_vecs = spam_data1.text.str.split().map(lambda x: get_doc_vec(x, spam_model))
spam_vecs

0       [-0.147819, 0.0350949, 0.0436575, 0.171935, -0...
1       [-0.164407, 0.0420122, 0.0498686, 0.185454, -0...
2       [-0.139702, 0.0308809, 0.0405548, 0.167575, -0...
3       [-0.207017, 0.0507612, 0.0632579, 0.239134, -0...
4       [-0.151833, 0.0361443, 0.0465693, 0.173738, -0...
5       [-0.149594, 0.0332996, 0.0449054, 0.175422, -0...
6       [-0.16496, 0.0400511, 0.0530456, 0.195745, -0....
7       [-0.0989628, 0.0221459, 0.02983, 0.115438, -0....
8       [-0.143616, 0.0357231, 0.0452015, 0.174036, -0...
9       [-0.178312, 0.0394015, 0.0546452, 0.217981, -0...
10      [-0.149825, 0.0352149, 0.0439142, 0.175553, -0...
11      [-0.162345, 0.037339, 0.0484078, 0.191343, -0....
12      [-0.185733, 0.0395844, 0.0555984, 0.227221, -0...
13      [-0.121024, 0.026502, 0.0361365, 0.13955, -0.0...
14      [-0.0876439, 0.0223248, 0.0282904, 0.0999922, ...
15      [-0.139797, 0.0287392, 0.0408726, 0.163992, -0...
16      [-0.175126, 0.0415515, 0.0576493, 0.208694, -0...
17      [-0.13

Some of the documents have no good words in them.  Let's drop them from our dataset, but before we do add them back into the original DataFrame:

In [28]:
# Add to dataframe
spam_data1['vecs'] = spam_vecs
# Drop the bad docs
spam_data1 = spam_data1.dropna()
spam_data1.shape

(5517, 3)

Now let's just convert the format that we have into a final DataFrame with 100 features and 1 label for use in our document classification task:

In [29]:
# Create a Numpy array of the document vectors
spam_np_vecs = np.zeros((len(spam_data1), 100))
for i, vec in enumerate(spam_data1.vecs):
    spam_np_vecs[i, :] = vec
    
# Combine the full dataframe with the labels
spam_w2v_data = pd.concat([spam_data1.reset_index().label, pd.DataFrame(spam_np_vecs)], axis=1)

Now we've arrived at a familiar point.  We have features and 1 label and we can use them to perform text classification.  As you already know how to do this, it is left for the exercises.

# Word2Vec Exercises
In these exercises, we'll finish experimenting with the spam data by using your word2vec vectors for text classification.  You will classify spam/ham using **both** your vectors and the pretrained Google vectors.  Then we'll move on to a richer dataset for you to perform the entire pipeline for Text Classification with Word2Vec.

## Spam Classification with Word2Vec

### Question 1
Use the spam word2vec dataframe from above to train and evaluate any classification algorithm for spam/ham.  **Hint**: Try a K-Nearest Neighbors Classifier.

Use the Google vectors from the beginning of this notebook to generate a new spam word2vec dataframe for text classification.

Build yet another spam/ham classification model using the word2vec vectors from Google.

Compare the performance of the 2 classifiers.

#### Trained Word2Vec Spam Classifier

In [30]:
## Training a Classifier with our own trained vectors
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Split the data
X_spam1 = spam_w2v_data.iloc[:, 1:]
y_spam1 = spam_w2v_data.label
X_train_spam1, X_test_spam1, y_train_spam1, y_test_spam1 = train_test_split(X_spam1, y_spam1, test_size=0.3)

# Train a KNN or Logistic Regression classifier
est = KNeighborsClassifier(algorithm='brute', metric='cosine')
# est = LogisticRegression()
est.fit(X_train_spam1, y_train_spam1)
est.score(X_test_spam1, y_test_spam1)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.96859903381642509

#### Document Vectors from Google Vectors

In [31]:
# Make a copy of the spam dataframe for the Google work
spam_data2 = spam_data.copy()

# Retrieve the document vectors based on google word vectors
spam_google_vecs = spam_data2.text.str.split().map(lambda x: get_doc_vec(x, google_model))

# Add to dataframe
spam_data2['vecs'] = spam_google_vecs

# Drop the bad docs
spam_data2 = spam_data2.dropna()

# Create a Numpy array of the document vectors
spam_np_vecs = np.zeros((len(spam_data2), 300))
for i, vec in enumerate(spam_data2.vecs):
    spam_np_vecs[i, :] = vec
    
# Combine the full dataframe with the labels
spam_google_data = pd.concat([spam_data2.reset_index().label, pd.DataFrame(spam_np_vecs)], axis=1)

#### Google Word2Vec Spam Classifier

In [32]:
## Training a Classifier with Google's vectors
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Split the data
X_spam2 = spam_google_data.iloc[:, 1:]
y_spam2 = spam_google_data.label
X_train_spam2, X_test_spam2, y_train_spam2, y_test_spam2 = train_test_split(X_spam2, y_spam2, test_size=0.3)

# Train a KNN or Logistic Regression classifier
est = KNeighborsClassifier(algorithm='brute', metric='cosine')
# est = LogisticRegression()
est.fit(X_train_spam2, y_train_spam2)
est.score(X_test_spam2, y_test_spam2)

0.97661870503597126

You should see that there is not much difference in performance between your vectors and Google's for this task (though Google's is better).  **However**, this is a fairly trivial problem/dataset, and we were already performing quite well on it.  So let's try something harder!

## 20 Newsgroups Classification with Word2Vec

### The Data

We will be using a portion of a data set containing approximately 20,000 posts partitioned evenly across 20 different newsgroups. This data set is quite famous. We will be using a sample of this data set, containing 5 topics and about 3,000 posts.

We will begin by loading the data.

In [33]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from accessory_functions import preprocess_series_text, nltk_path

topic_list = ['sci.space', 'comp.sys.mac.hardware', 'rec.autos',
              'rec.sport.baseball', 'sci.med']

# Retrieve the data into a DataFrame
dataset = fetch_20newsgroups(shuffle=True, random_state=1, data_home='../Data',
                             categories=topic_list,
                             remove=('headers', 'footers', 'quotes'))
ng_data = pd.DataFrame(dataset['data'], columns=['text'])
ng_data['label'] = dataset['target']

# Preprocess the text
ng_data['text'] = preprocess_series_text(ng_data.text, nltk_path=nltk_path)


print(len(ng_data))
ng_data.head()

2956


,text,label
0,otoh u get lucky unplugged replugged scsi adb ...,0
1,yes everyone else may wonder fred well would o...,4
2,umm perhaps could explain right talk,4
3,like alomar like differ opinion city likely po...,2
4,wow know uranus long way think far away,4


### Question 2
Train a Word2Vec model to generate word vectors from the 20 Newsgroups data.

Use your Word2Vec model to generate document vectors from these word vectors.

Combine these vectors with the 20 Newsgroups class labels to create a DataFrame for classification.

Train a classification model for these five 20 Newsgroups classes and evaluate its performance.  **Hint**: Try a K-Nearest Neighbors Model.

#### Training Word2Vec on 20 Newsgroups

In [34]:
from gensim.models import Word2Vec
# Generate sentences for training word2vec
sentences = ng_data.text.str.split()
# Train a Word2Vec model
ng_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

# Did it work?
ng_model.most_similar('baseball')

[('brave', 0.9996107220649719),
 ('pitcher', 0.9995369911193848),
 ('offense', 0.999420166015625),
 ('stats', 0.9993765354156494),
 ('chance', 0.9992481470108032),
 ('sign', 0.9991667866706848),
 ('hitter', 0.9991663694381714),
 ('lopez', 0.9991459250450134),
 ('throw', 0.9991354942321777),
 ('ball', 0.9991125464439392)]

#### Document Vectors from Word Vectors

In [35]:
# Make a copy of the spam dataframe for the Google work
ng_data1 = ng_data.copy()

# Retrieve the document vectors based on google word vectors
ng_vecs = ng_data1.text.str.split().map(lambda x: get_doc_vec(x, ng_model))

# Add to dataframe
ng_data1['vecs'] = ng_vecs

# Drop the bad docs
ng_data1 = ng_data1.dropna()

# Create a Numpy array of the document vectors
ng_np_vecs = np.zeros((len(ng_data1), 100))
for i, vec in enumerate(ng_data1.vecs):
    ng_np_vecs[i, :] = vec
    
# Combine the full dataframe with the labels
ng_w2v_data = pd.concat([ng_data1.reset_index().label, pd.DataFrame(ng_np_vecs)], axis=1)

#### Trained Word2Vec 20 Newsgroups Classifier

In [36]:
## Training a Classifier with our own trained vectors
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Split the data
X_ng1 = ng_w2v_data.iloc[:, 1:]
y_ng1 = ng_w2v_data.label
X_train_ng1, X_test_ng1, y_train_ng1, y_test_ng1 = train_test_split(X_ng1, y_ng1, test_size=0.3)

# Train a KNN or Logistic Regression classifier
est = KNeighborsClassifier(algorithm='brute', metric='cosine')
# est = LogisticRegression()
est.fit(X_train_ng1, y_train_ng1)
est.score(X_test_ng1, y_test_ng1)

0.61032863849765262

### Question 3
Use the Google vectors to generate document vectors for the 20 Newsgroups data.

Combine these vectors with the 20 Newsgroups class labels to create a DataFrame for classification.

Train a classification model for these five 20 Newsgroups classes and evaluate its performance.  Train a classification model for these five 20 Newsgroups classes and evaluate its performance.  **Hint**: Try a K-Nearest Neighbors Model.

Note the performance of the Google vectors vs your own Word2Vec training.

#### 20 Newsgroups Document Vectors from Google Word Vectors

In [37]:
# Make a copy of the spam dataframe for the Google work
ng_data2 = ng_data.copy()

# Retrieve the document vectors based on google word vectors
ng_google_vecs = ng_data2.text.str.split().map(lambda x: get_doc_vec(x, google_model))

# Add to dataframe
ng_data2['vecs'] = ng_google_vecs

# Drop the bad docs
ng_data2 = ng_data2.dropna()

# Create a Numpy array of the document vectors
ng_np_vecs = np.zeros((len(ng_data2), 300))
for i, vec in enumerate(ng_data2.vecs):
    ng_np_vecs[i, :] = vec
    
# Combine the full dataframe with the labels
ng_google_data = pd.concat([ng_data2.reset_index().label, pd.DataFrame(ng_np_vecs)], axis=1)

#### Google Word2Vec Newsgroups Classifier

In [38]:
## Training a Classifier with Google's vectors
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Split the data
X_ng2 = ng_google_data.iloc[:, 1:]
y_ng2 = ng_google_data.label
X_train_ng2, X_test_ng2, y_train_ng2, y_test_ng2 = train_test_split(X_ng2, y_ng2, test_size=0.3)

# Train a KNN or Logistic Regression classifier
est = KNeighborsClassifier(algorithm='brute', metric='cosine')
# est = LogisticRegression()
est.fit(X_train_ng2, y_train_ng2)
est.score(X_test_ng2, y_test_ng2)

0.87002341920374704

**WOW!**  Look how much better the Google vectors did.  This should demonstrate how valuable a good set of word vectors can be.

### Question 4
Recall TFIDF Vectorizer from the previous exercises.  Generate a DataFrame for classification using `TfidfVectorizer` and the 20 Newsgroups subset.

Train a classification model on the TFIDF results using either Logistic Regression or Naive Bayes.

Compare the results between your TFIDF model, trained word2vec, and Google word2vec.  Which does best?

#### 20 Newsgroups Classification with TFIDF

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Generate TFIDF Vectors
tfidf = TfidfVectorizer()
ng_tfidf = tfidf.fit_transform(ng_data['text'])

# Split the data
X_ng_tfidf = ng_tfidf
y_ng_tfidf = ng_data['label']
X_ng_tfidf_train, X_ng_tfidf_test, y_ng_tfidf_train, y_ng_tfidf_test = train_test_split(X_ng_tfidf, y_ng_tfidf, test_size=0.3)

# Train a Logistic Regression
lr = LogisticRegression()
lr.fit(X_ng_tfidf_train, y_ng_tfidf_train)

# Evaluate
lr.score(X_ng_tfidf_test, y_ng_tfidf_test)

0.88500563697857948

Well, in this case the TFIDF did great, on par or better than word2vec.  However, TFIDF will often do very well in cases where certain trigger words are highly useful in distinguishing between classes.  If you look through the different document categories that we chose, they are ***highly*** different.  In cases where the differences between classes are more subtle, you should expect a Word2Vec model to strongly outperform TFIDF.  With more time, you can try downloading all 20 categories from 20 Newsgroups and seeing how the various models perform.  The full set has classes that are much tougher to tease apart than the relatively disjoint subset that we used.